In [ ]:
from transformers import PreTrainedTokenizerFast 

tokenizer = PreTrainedTokenizerFast.from_pretrained("charanhu/kannada-tokenizer")

text = train_data['tgt']
src_text = train_data['src']

print(text[2])
print(src_text[2])

In [ ]:
for i, s_txt in enumerate(text):
    encoding = tokenizer.encode(s_txt) 
    tokens = tokenizer.convert_ids_to_tokens(encoding) 
    decoded_text = tokenizer.decode(encoding) 

    # print(f"original text : {s_txt}") 
    # print(f"encoded text: {encoding}")
    # print(f"tokens: {tokens}") 
    # print(f"decoded_text: {decoded_text}") 

In [1]:
from transformers import AutoModel, AutoTokenizer
import itertools
import torch

# load model
model = AutoModel.from_pretrained("aneuraz/awesome-align-with-co")
tokenizer = AutoTokenizer.from_pretrained("aneuraz/awesome-align-with-co")

# model parameters
align_layer = 8
threshold = 1e-3

# define inputs
src = 'the universe is vast and who knows how many species are within it.'
tgt = 'ವಿಶ್ವವು ವಿಶಾಲವಾಗಿದೆ ಮತ್ತು ಅದರೊಳಗೆ ಎಷ್ಟು ಜಾತಿಗಳಿವೆ ಎಂದು ಯಾರಿಗೆ ತಿಳಿದಿದೆ.'

# pre-processing
sent_src, sent_tgt = src.strip().split(), tgt.strip().split()

print("="*50)
print("original words:")
print("="*50)
print("Source words:", sent_src)
print("Target words:", sent_tgt)
print()

token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]

print("="*50)
print("tokenization (Word -> Subwords):")
print("="*50)
print("src tokenization:")
for i, (word, tokens) in enumerate(zip(sent_src, token_src)):
    print(f"Word {i}: '{word}' -> {tokens}")

print("\tgt tokenization:")
for i, (word, tokens) in enumerate(zip(sent_tgt, token_tgt)):
    print(f"Word {i}: '{word}' -> {tokens}")
print()

flat_tokens_src = list(itertools.chain(*token_src))
flat_tokens_tgt = list(itertools.chain(*token_tgt))

print("="*50)
print("="*50)
print("src subwords :")
for i, token in enumerate(flat_tokens_src):
    print(f"Subword {i}: '{token}'")

print("\n tgt subwords:")
for i, token in enumerate(flat_tokens_tgt):
    print(f"Subword {i}: '{token}'")
print()

# Convert to IDs
wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]

print("="*50)
print("token ids (Word level):")
print("="*50)
print("src token ids:")
for i, (word, tokens, ids) in enumerate(zip(sent_src, token_src, wid_src)):
    print(f"Word {i}: '{word}' -> tokens: {tokens} -> ids: {ids}")

print("\tgt token ids:")
for i, (word, tokens, ids) in enumerate(zip(sent_tgt, token_tgt, wid_tgt)):
    print(f"Word {i}: '{word}' -> tokens: {tokens} -> ids: {ids}")
print()

ids_src = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids']
ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']

print("="*50)
print("final model input (with special tokens):")
print("="*50)
print("SOURCE INPUT IDs:", ids_src.squeeze().tolist())
print("SOURCE TOKENS:", tokenizer.convert_ids_to_tokens(ids_src.squeeze().tolist()))

print("\target input ids:", ids_tgt.squeeze().tolist())
print("tgt tokens:", tokenizer.convert_ids_to_tokens(ids_tgt.squeeze().tolist()))
print()

sub2word_map_src = []
sub2word_map_tgt = []

print("="*50)
print("subword to word mapping")
print("="*50)
print("src mapping ")
subword_idx_src = 0
for word_idx, word_tokens in enumerate(token_src):
    print(f"Word {word_idx} ('{sent_src[word_idx]}'):")
    for token in word_tokens:
        print(f"  Subword {subword_idx_src}: '{token}' -> maps to word {word_idx}")
        sub2word_map_src.append(word_idx)
        subword_idx_src += 1

print(f"\nSource sub2word_map: {sub2word_map_src}")

subword_idx_tgt = 0
for word_idx, word_tokens in enumerate(token_tgt):
    print(f"Word {word_idx} ('{sent_tgt[word_idx]}'):")
    for token in word_tokens:
        print(f"  Subword {subword_idx_tgt}: '{token}' -> maps to word {word_idx}")
        sub2word_map_tgt.append(word_idx)
        subword_idx_tgt += 1


# alignment
model.eval()
with torch.no_grad():
    out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
    out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
    dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))
    softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
    softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)
    softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
align_words = set()

print("="*50)
print("="*50)
for i, j in align_subwords:
    src_subword = flat_tokens_src[i.item()]
    tgt_subword = flat_tokens_tgt[j.item()]
    src_word_idx = sub2word_map_src[i.item()]
    tgt_word_idx = sub2word_map_tgt[j.item()]
    src_word = sent_src[src_word_idx]
    tgt_word = sent_tgt[tgt_word_idx]
    
    print(f"Subword {i.item()} '{src_subword}' (from word {src_word_idx} '{src_word}') <-> "
          f"Subword {j.item()} '{tgt_subword}' (from word {tgt_word_idx} '{tgt_word}')")
    
    align_words.add((src_word_idx, tgt_word_idx))

print(f"\nWORD-LEVEL ALIGNMENTS:")
for src_idx, tgt_idx in sorted(align_words):
    print(f"Word {src_idx} '{sent_src[src_idx]}' <-> Word {tgt_idx} '{sent_tgt[tgt_idx]}'")

print(f"\nFinal alignment set: {align_words}")

config.json: 0.00B [00:00, ?B/s]

2025-07-01 14:22:58.823366: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751379779.014786      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751379779.074126      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at aneuraz/awesome-align-with-co and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ORIGINAL WORDS:
Source words: ['the', 'universe', 'is', 'vast', 'and', 'who', 'knows', 'how', 'many', 'species', 'are', 'within', 'it.']
Target words: ['ವಿಶ್ವವು', 'ವಿಶಾಲವಾಗಿದೆ', 'ಮತ್ತು', 'ಅದರೊಳಗೆ', 'ಎಷ್ಟು', 'ಜಾತಿಗಳಿವೆ', 'ಎಂದು', 'ಯಾರಿಗೆ', 'ತಿಳಿದಿದೆ.']

TOKENIZATION (Word -> Subwords):
SOURCE TOKENIZATION:
Word 0: 'the' -> ['the']
Word 1: 'universe' -> ['universe']
Word 2: 'is' -> ['is']
Word 3: 'vast' -> ['vast']
Word 4: 'and' -> ['and']
Word 5: 'who' -> ['who']
Word 6: 'knows' -> ['knows']
Word 7: 'how' -> ['how']
Word 8: 'many' -> ['many']
Word 9: 'species' -> ['species']
Word 10: 'are' -> ['are']
Word 11: 'within' -> ['within']
Word 12: 'it.' -> ['it', '.']

TARGET TOKENIZATION:
Word 0: 'ವಿಶ್ವವು' -> ['ವ', '##ಿ', '##ಶ್', '##ವ', '##ವು']
Word 1: 'ವಿಶಾಲವಾಗಿದೆ' -> ['ವ', '##ಿ', '##ಶ', '##ಾಲ', '##ವಾಗಿದೆ']
Word 2: 'ಮತ್ತು' -> ['ಮತ್ತು']
Word 3: 'ಅದರೊಳಗೆ' -> ['ಅದರ', '##ೊ', '##ಳ', '##ಗೆ']
Word 4: 'ಎಷ್ಟು' -> ['ಎ', '##ಷ್ಟು']
Word 5: 'ಜಾತಿಗಳಿವೆ' -> ['ಜ', '##ಾತ', '##ಿ', '##ಗಳ', '##ಿವೆ']
Word 6: 'ಎಂದ

In [1]:
from transformers import AutoModel, AutoTokenizer, PreTrainedTokenizerFast
import itertools
import torch
from collections import defaultdict, Counter

align_model = AutoModel.from_pretrained("aneuraz/awesome-align-with-co")
align_tokenizer = AutoTokenizer.from_pretrained("aneuraz/awesome-align-with-co")

kn_tokenizer = PreTrainedTokenizerFast.from_pretrained("charanhu/kannada-tokenizer")

align_layer = 8
threshold = 1e-3

all_token_pairs = []  
pair_frequency = Counter() 

def get_alignments(src_text, tgt_text):
    """
    Get word alignments using awesome-align model
    Returns: set of (src_word_idx, tgt_word_idx) tuples
    """
    sent_src = src_text.strip().split()
    sent_tgt = tgt_text.strip().split()

    # awesome aligner for mapping src to tgt word 
    token_src = [align_tokenizer.tokenize(word) for word in sent_src]
    token_tgt = [align_tokenizer.tokenize(word) for word in sent_tgt]
    
    # tokens to IDs
    wid_src = [align_tokenizer.convert_tokens_to_ids(x) for x in token_src]
    wid_tgt = [align_tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
    
    ids_src = align_tokenizer.prepare_for_model(
        list(itertools.chain(*wid_src)), 
        return_tensors='pt', 
        model_max_length=align_tokenizer.model_max_length, 
        truncation=True
    )['input_ids']
    
    ids_tgt = align_tokenizer.prepare_for_model(
        list(itertools.chain(*wid_tgt)), 
        return_tensors='pt', 
        truncation=True, 
        model_max_length=align_tokenizer.model_max_length
    )['input_ids']
    
    # Create subword to word mapping
    sub2word_map_src = []
    for i, word_list in enumerate(token_src):
        sub2word_map_src += [i for x in word_list]
    
    sub2word_map_tgt = []
    for i, word_list in enumerate(token_tgt):
        sub2word_map_tgt += [i for x in word_list]
    
    # Get alignments
    align_model.eval()
    with torch.no_grad():
        out_src = align_model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
        out_tgt = align_model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
        
        dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))
        softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
        softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)
        softmax_inter = (softmax_srctgt > threshold) * (softmax_tgtsrc > threshold)
    
    align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
    
    # word level  alignments 
    align_words = set()
    for i, j in align_subwords:
        align_words.add((sub2word_map_src[i], sub2word_map_tgt[j]))
    
    return align_words, sent_src, sent_tgt

for idx, (src_text, tgt_text) in enumerate(zip(train_data["src"], train_data["tgt"])):
    if idx % 100 == 0: 
        print(f"Processed {idx} sentences...")
    
    try:
        alignments, src_words, tgt_words = get_alignments(src_text, tgt_text)
        
        en_tokens = align_tokenizer.tokenize(src_text)  
        
        kn_tokens = align_tokenizer.tokenize(tgt_text)    
        
        # create token pairs from alignments
        for src_idx, tgt_idx in alignments:
            if src_idx < len(en_tokens) and tgt_idx < len(kn_tokens):
                en_token = en_tokens[src_idx]
                kn_token = kn_tokens[tgt_idx] if tgt_idx < len(kn_tokens) else tgt_words[tgt_idx]
                
                # create token pair
                token_pair = (en_token, kn_token)
                all_token_pairs.append(token_pair)
                
                # ct frequency
                pair_frequency[token_pair] += 1
        
        if idx < 3:
            print(f"\nExample {idx + 1}:")
            print(f"English: {src_text}")
            print(f"Kannada: {tgt_text}")
            print(f"Alignments: {alignments}")
            print(f"Token pairs: {[(en_tokens[i], kn_tokens[j] if j < len(kn_tokens) else tgt_words[j]) for i, j in alignments if i < len(en_tokens)]}")
            
    except Exception as e:
        print(f"Error processing sentence {idx}: {e}")
        continue

print(f"\nAlignment processing complete!")
print(f"Total unique token pairs: {len(pair_frequency)}")
print(f"Total alignment instances: {sum(pair_frequency.values())}")

# Display most frequent token pairs
print("\nMost frequent token pairs:")
for (en_token, kn_token), freq in pair_frequency.most_common(20):
    print(f"'{en_token}' <-> '{kn_token}': {freq} times")

# Filter by minimum frequency threshold
min_frequency = 5
filtered_pairs = {pair: freq for pair, freq in pair_frequency.items() if freq >= min_frequency}
print(f"\nToken pairs with frequency >= {min_frequency}: {len(filtered_pairs)}")

torch.save(pair_frequency, 'kannada_english_token_pairs.pt')
print("tok pair saved")

config.json: 0.00B [00:00, ?B/s]

2025-07-03 14:00:47.202271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751551247.394399      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751551247.452753      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at aneuraz/awesome-align-with-co and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Processing sentence pairs...


NameError: name 'train_data' is not defined

In [1]:
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
import itertools
import torch
import json
import pickle
from tqdm import tqdm
import os
from typing import List, Tuple, Set, Dict, Any
import gc

class WordAligner:
    def __init__(self, model_name="aneuraz/awesome-align-with-co", align_layer=8, threshold=1e-3, use_gpu=True):
        """Initialize the word aligner with model and parameters."""
        print("Loading model and tokenizer...")
        self.device = torch.device("cuda" if torch.cuda.is_available() and use_gpu else "cpu")
        print(f"Using device: {self.device}")
        
        self.model = AutoModel.from_pretrained(model_name, torch_dtype=torch.float16).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.align_layer = align_layer
        self.threshold = threshold
        self.model.eval()
        
        
        self.use_amp = use_gpu and torch.cuda.is_available()
        if self.use_amp:
            print("mixed precision for faster inference")
        
        print("model loaded")
    
    def align_sentence_pair(self, src_text: str, tgt_text: str) -> Dict[str, Any]:
        """
            src_words: list of source words
            tgt_words: list of target words
            word_alignments: set of (src_idx, tgt_idx) tuples
            subword_alignments: list of detailed subword alignment info
        """
            sent_src = src_text.strip().split()
            sent_tgt = tgt_text.strip().split()
            
            if not sent_src or not sent_tgt:
                return None
            
            token_src = [self.tokenizer.tokenize(word) for word in sent_src]
            token_tgt = [self.tokenizer.tokenize(word) for word in sent_tgt]
            
            if not all(token_src) or not all(token_tgt):
                return None
            
            # flatten token lists 
            flat_tokens_src = list(itertools.chain(*token_src))
            flat_tokens_tgt = list(itertools.chain(*token_tgt))
            
            wid_src = [self.tokenizer.convert_tokens_to_ids(x) for x in token_src]
            wid_tgt = [self.tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
            
            ids_src = self.tokenizer.prepare_for_model(
                list(itertools.chain(*wid_src)), 
                return_tensors='pt', 
                model_max_length=self.tokenizer.model_max_length, 
                truncation=True
            )['input_ids'].to(self.device)
            
            ids_tgt = self.tokenizer.prepare_for_model(
                list(itertools.chain(*wid_tgt)), 
                return_tensors='pt', 
                truncation=True, 
                model_max_length=self.tokenizer.model_max_length
            )['input_ids'].to(self.device)
            
            # subword to word mappiing 
            sub2word_map_src = []
            for word_idx, word_tokens in enumerate(token_src):
                sub2word_map_src.extend([word_idx] * len(word_tokens))
            
            sub2word_map_tgt = []
            for word_idx, word_tokens in enumerate(token_tgt):
                sub2word_map_tgt.extend([word_idx] * len(word_tokens))
            
            # mixed precision 
            with torch.no_grad():
                if self.use_amp:
                    with torch.cuda.amp.autocast():
                        out_src = self.model(ids_src.unsqueeze(0), output_hidden_states=True)[2][self.align_layer][0, 1:-1]
                        out_tgt = self.model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][self.align_layer][0, 1:-1]
                        dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))
                else:
                    out_src = self.model(ids_src.unsqueeze(0), output_hidden_states=True)[2][self.align_layer][0, 1:-1]
                    out_tgt = self.model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][self.align_layer][0, 1:-1]
                    dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))
                
                softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
                softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)
                softmax_inter = (softmax_srctgt > self.threshold) * (softmax_tgtsrc > self.threshold)
            
            align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
            align_words = set()
            subword_alignments = []
            
            for i, j in align_subwords:
                i_idx, j_idx = i.item(), j.item()
                
                if i_idx >= len(sub2word_map_src) or j_idx >= len(sub2word_map_tgt):
                    continue
                
                src_subword = flat_tokens_src[i_idx]
                tgt_subword = flat_tokens_tgt[j_idx]
                src_word_idx = sub2word_map_src[i_idx]
                tgt_word_idx = sub2word_map_tgt[j_idx]
                
                subword_alignments.append({
                    'src_subword_idx': i_idx,
                    'tgt_subword_idx': j_idx,
                    'src_subword': src_subword,
                    'tgt_subword': tgt_subword,
                    'src_word_idx': src_word_idx,
                    'tgt_word_idx': tgt_word_idx,
                    'src_word': sent_src[src_word_idx],
                    'tgt_word': sent_tgt[tgt_word_idx]
                })
                
                align_words.add((src_word_idx, tgt_word_idx))
            
            return {
                'src_text': src_text,
                'tgt_text': tgt_text,
                'src_words': sent_src,
                'tgt_words': sent_tgt,
                'word_alignments': list(align_words),
                'subword_alignments': subword_alignments,
                'alignment_count': len(align_words)
            }
            

def process_dataset_batch(dataset, aligner, output_dir="alignment_output", batch_size=4000, max_samples=None):
    """Process the entire dataset in batches and save alignments."""
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Determine total samples to process
    total_samples = len(dataset) if max_samples is None else min(max_samples, len(dataset))
    print(f"Processing {total_samples} sentence pairs...")
    
    # Process in batches
    batch_num = 0
    alignments = []
    successful_alignments = 0
    failed_alignments = 0
    
    for i in tqdm(range(0, total_samples, batch_size), desc="Processing batches"):
        batch_alignments = []
        
        # Process current batch
        end_idx = min(i + batch_size, total_samples)
        for j in range(i, end_idx):
            sample = dataset[j]
            src_text = sample['src']  # English
            tgt_text = sample['tgt']  # Kannada
            
            alignment_result = aligner.align_sentence_pair(src_text, tgt_text)
            
            if alignment_result is not None:
                alignment_result['sample_idx'] = j
                batch_alignments.append(alignment_result)
                successful_alignments += 1
            else:
                failed_alignments += 1
        
        # Save batch to disk
        if batch_alignments:
            batch_file = os.path.join(output_dir, f"alignments_batch_{batch_num:04d}.pkl")
            with open(batch_file, 'wb') as f:
                pickle.dump(batch_alignments, f)
            
            # Also save as JSON for human readability (smaller sample)
            if len(batch_alignments) > 0:
                json_sample = batch_alignments[:10]  # Save first 10 as JSON for inspection
                json_file = os.path.join(output_dir, f"alignments_batch_{batch_num:04d}_sample.json")
                with open(json_file, 'w', encoding='utf-8') as f:
                    json.dump(json_sample, f, ensure_ascii=False, indent=2)
        
        alignments.extend(batch_alignments)
        batch_num += 1
        
        # Clear memory periodically
        if batch_num % 10 == 0:
            gc.collect()
            print(f"Processed {batch_num} batches. Success: {successful_alignments}, Failed: {failed_alignments}")
    
    # Save final statistics
    stats = {
        'total_processed': total_samples,
        'successful_alignments': successful_alignments,
        'failed_alignments': failed_alignments,
        'success_rate': successful_alignments / total_samples * 100,
        'total_batches': batch_num
    }
    
    stats_file = os.path.join(output_dir, "alignment_stats.json")
    with open(stats_file, 'w') as f:
        json.dump(stats, f, indent=2)
    
    print(f"\nAlignment completed!")
    print(f"Successful alignments: {successful_alignments}")
    print(f"Failed alignments: {failed_alignments}")
    print(f"Success rate: {stats['success_rate']:.2f}%")
    print(f"Results saved in: {output_dir}")
    
    return alignments, stats

def load_alignments_from_batches(output_dir="alignment_output"):
    """Load all alignment batches from disk."""
    alignments = []
    batch_files = sorted([f for f in os.listdir(output_dir) if f.startswith("alignments_batch_") and f.endswith(".pkl")])
    
    print(f"Loading {len(batch_files)} batch files...")
    for batch_file in tqdm(batch_files):
        with open(os.path.join(output_dir, batch_file), 'rb') as f:
            batch_alignments = pickle.load(f)
            alignments.extend(batch_alignments)
    
    return alignments

def create_translation_training_data(alignments, output_file="translation_training_data.json"):
    """Create training data optimized for translation models."""
    training_data = []
    
    for alignment in alignments:
        # Basic sentence pair
        training_sample = {
            'src': alignment['src_text'],
            'tgt': alignment['tgt_text'],
            'word_alignments': alignment['word_alignments'],
            'alignment_count': alignment['alignment_count']
        }
        
        # Add aligned word pairs for additional supervision
        aligned_pairs = []
        for src_idx, tgt_idx in alignment['word_alignments']:
            aligned_pairs.append({
                'src_word': alignment['src_words'][src_idx],
                'tgt_word': alignment['tgt_words'][tgt_idx],
                'src_idx': src_idx,
                'tgt_idx': tgt_idx
            })
        
        training_sample['aligned_word_pairs'] = aligned_pairs
        training_data.append(training_sample)
    
    # Save training data
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(training_data, f, ensure_ascii=False, indent=2)
    
    print(f"Training data saved to: {output_file}")
    return training_data

# Main execution
if __name__ == "__main__":
    # Load dataset
    print("Loading Samanantar Kannada dataset...")
    ds = load_dataset("ai4bharat/samanantar", "kn")
    train_data = ds['train']
    print(f"Dataset loaded with {len(train_data)} samples")
    
    # Initialize aligner
    aligner = WordAligner()

    # temp testing purposes 
    max_samples = 400000 
    
    alignments, stats = process_dataset_batch(
        train_data, 
        aligner, 
        output_dir="/kaggle/working/samanantar_alignments",
        batch_size=2000,  # Increased batch size for GPU
        max_samples=max_samples
    )
    
    # Create translation training data
    training_data = create_translation_training_data(
        alignments, 
        "/kaggle/working/samanantar_kn_translation_training.json"
    )
    
    print(f"\nProcessing complete! Generated {len(training_data)} training samples.")
    
    # Example of how to use the alignments
    if alignments:
        print("\nExample alignment:")
        example = alignments[0]
        print(f"Source: {example['src_text']}")
        print(f"Target: {example['tgt_text']}")
        print(f"Word alignments: {example['word_alignments']}")
        print("Aligned word pairs:")
        for src_idx, tgt_idx in example['word_alignments']:
            print(f"  '{example['src_words'][src_idx]}' <-> '{example['tgt_words'][tgt_idx]}'")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/242M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/242M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4093524 [00:00<?, ? examples/s]

...


Processing batches:  95%|█████████▌| 190/200 [2:15:09<07:10, 43.02s/it]

Processed 190 batches. Success: 378678, Failed: 1322



Processing batches: 100%|██████████| 200/200 [2:22:15<00:00, 42.68s/it]

Processed 200 batches. Success: 398614, Failed: 1386

Alignment completed!
Successful alignments: 398614
Failed alignments: 1386
Success rate: 99.65%
Results saved in: /kaggle/working/samanantar_alignments


Training data saved to: /kaggle/working/samanantar_kn_translation_training.json

Processing complete! Generated 398614 training samples.

Example alignment:
Source: Hes a scientist.
Target: ಇವರು ಸಂಶೋಧಕ ಸ್ವಭಾವದವರು.
Word alignments: [(1, 2), (2, 2), (2, 1), (0, 0)]
Aligned word pairs:
  'a' <-> 'ಸ್ವಭಾವದವರು.'
  'scientist.' <-> 'ಸ್ವಭಾವದವರು.'
  'scientist.' <-> 'ಸಂಶೋಧಕ'
  'Hes' <-> 'ಇವರು'


In [ ]:
import json 
with open ('/kaggle/working/samanantar_kn_translation_training.json', 'r') as file:
    data = json.load(file)
    print(data)
    print("\nPretty-printed JSON:")
    print(json.dumps(data, indent=4))


In [1]:
!pip install transformers torch
!pip install -U datasets

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_len=5000):
        super().__init__()
        pe = torch.zeros(max_seq_len, d_model)
        position = torch.arange(0, max_seq_len).unsqueeze(1).float()
        
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                           -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model % 2 == 1:
            pe[:, 1::2] = torch.cos(position * div_term[:-1])
        else:
            pe[:, 1::2] = torch.cos(position * div_term)
        
        # self.register_buffer('pe', pe.unsqueeze(0))
    
    def forward(self, x):
        # input shape: [batch_size, seq_len, d_model]
        # pe shape: [1, max_seq_len, d_model]
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)
        
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        seq_len = query.size(1)
        
        Q = self.q_linear(query).view(batch_size, seq_len, self.n_heads, self.head_dim).transpose(1, 2)
        K = self.k_linear(key).view(batch_size, key.size(1), self.n_heads, self.head_dim).transpose(1, 2)
        V = self.v_linear(value).view(batch_size, value.size(1), self.n_heads, self.head_dim).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
            
        attention_weights = F.softmax(scores, dim=-1)
        context = torch.matmul(attention_weights, V)
        
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        
        return self.out(context), attention_weights

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        return self.linear2(F.relu(self.linear1(x)))

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attention = MultiHeadAttention(d_model, n_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        attn_output, _ = self.self_attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attention = MultiHeadAttention(d_model, n_heads)
        self.cross_attention = MultiHeadAttention(d_model, n_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, encoder_output, self_mask=None, cross_mask=None):
        self_attn_output, _ = self.self_attention(x, x, x, self_mask)
        x = self.norm1(x + self.dropout(self_attn_output))
        
        cross_attn_output, cross_attention_weights = self.cross_attention(x, encoder_output, encoder_output, cross_mask)
        x = self.norm2(x + self.dropout(cross_attn_output))
        
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        
        return x, cross_attention_weights

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff, max_seq_len, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_seq_len)
        self.layers = nn.ModuleList([EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)])
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask=None):
        x = self.embedding(src) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        
        for layer in self.layers:
            x = layer(x, src_mask)
            
        return x

class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff, max_seq_len, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_seq_len)
        self.layers = nn.ModuleList([DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)])
        self.output_projection = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, tgt, encoder_output, tgt_mask=None, src_mask=None):
        x = self.embedding(tgt) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        
        cross_attention_weights = []
        
        for layer in self.layers:
            x, cross_attn_weights = layer(x, encoder_output, tgt_mask, src_mask)
            cross_attention_weights.append(cross_attn_weights)
            
        output = self.output_projection(x)
        
        return output, cross_attention_weights

class EnglishKannadaTransformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, n_heads=8, 
                 n_encoder_layers=6, n_decoder_layers=6, d_ff=2048, max_seq_len=5000, dropout=0.1):
        super().__init__()
        
        self.encoder = TransformerEncoder(src_vocab_size, d_model, n_heads, n_encoder_layers, 
                                        d_ff, max_seq_len, dropout)
        self.decoder = TransformerDecoder(tgt_vocab_size, d_model, n_heads, n_decoder_layers, 
                                        d_ff, max_seq_len, dropout)
        
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        encoder_output = self.encoder(src, src_mask)
        decoder_output, cross_attention_weights = self.decoder(tgt, encoder_output, tgt_mask, src_mask)
        
        return decoder_output, cross_attention_weights
    
    def create_masks(self, src, tgt):
        device = src.device
        
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        
        # causal mask + padding mask
        tgt_len = tgt.size(1)
        
        tgt_mask = torch.tril(torch.ones(tgt_len, tgt_len, device=device)).unsqueeze(0).unsqueeze(0).bool()
        
        tgt_padding_mask = (tgt != 0).unsqueeze(1).unsqueeze(2)
        
        tgt_mask = tgt_mask & tgt_padding_mask
        
        return src_mask, tgt_mask
        
def calculate_alignment_loss(cross_attention_weights, alignment_matrix, lambda_align=0.1):
    
    last_layer_attention = cross_attention_weights[-1]  
    
    avg_attention = last_layer_attention.mean(dim=1) 
    
     # [batch, tgt_len, src_len]
    alignment_target = alignment_matrix.transpose(-2, -1).float() 
    
    # Calculate MSE loss
    alignment_loss = F.mse_loss(avg_attention, alignment_target)
    
    return lambda_align * alignment_loss

def calculate_total_loss(model_output, target, cross_attention_weights, alignment_matrix, lambda_align=0.1):
    # Translation loss (cross-entropy)
    vocab_size = model_output.size(-1)
    translation_loss = F.cross_entropy(model_output.view(-1, vocab_size), target.view(-1), ignore_index=0)
    
    # Alignment loss
    alignment_loss = calculate_alignment_loss(cross_attention_weights, alignment_matrix, lambda_align)
    
    total_loss = translation_loss + alignment_loss
    
    return total_loss, translation_loss, alignment_loss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
from collections import Counter
import numpy as np
from tqdm import tqdm

class TranslationDataset(Dataset):
    def __init__(self, json_file_path, src_vocab=None, tgt_vocab=None, max_len=128):
        with open(json_file_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        
        self.max_len = max_len
        
        if src_vocab is None or tgt_vocab is None:
            self.src_vocab, self.tgt_vocab = self.build_vocabularies()
        else:
            self.src_vocab = src_vocab
            self.tgt_vocab = tgt_vocab
            
        self.src_word2idx = {word: idx for idx, word in enumerate(self.src_vocab)}
        self.tgt_word2idx = {word: idx for idx, word in enumerate(self.tgt_vocab)}
        
        self.processed_data = self.process_data()
        
    def build_vocabularies(self):
        src_words = []
        tgt_words = []
        
        for item in self.data:
            src_words.extend(item['src'].split())
            tgt_words.extend(item['tgt'].split())
            
        src_counter = Counter(src_words)
        tgt_counter = Counter(tgt_words)
        
        src_vocab = ['<pad>', '<unk>', '<sos>', '<eos>'] + [word for word, _ in src_counter.most_common()]
        tgt_vocab = ['<pad>', '<unk>', '<sos>', '<eos>'] + [word for word, _ in tgt_counter.most_common()]
        
        return src_vocab, tgt_vocab
    
    def text_to_indices(self, text, word2idx, is_target=False):
        words = text.split()
        indices = []
        
        if is_target:
            indices.append(word2idx.get('<sos>', 1))
            
        for word in words:
            indices.append(word2idx.get(word, word2idx.get('<unk>', 1)))
            
        if is_target:
            indices.append(word2idx.get('<eos>', 1))
            
        return indices
    
    def create_alignment_matrix(self, src_text, tgt_text, word_alignments):
        src_words = src_text.split()
        tgt_words = tgt_text.split()
        
        src_len = len(src_words)
        tgt_len = len(tgt_words)
        
        alignment_matrix = np.zeros((src_len, tgt_len), dtype=np.float32)
        
        for src_idx, tgt_idx in word_alignments:
            if src_idx < src_len and tgt_idx < tgt_len:
                alignment_matrix[src_idx, tgt_idx] = 1.0
                
        return alignment_matrix
    
    def process_data(self):
        processed = []
        
        for item in self.data:
            src_indices = self.text_to_indices(item['src'], self.src_word2idx)
            tgt_indices = self.text_to_indices(item['tgt'], self.tgt_word2idx, is_target=True)
            
            alignment_matrix = self.create_alignment_matrix(
                item['src'], item['tgt'], item['word_alignments']
            )
            
            processed.append({
                'src': src_indices,
                'tgt': tgt_indices,
                'alignment': alignment_matrix
            })
            
        return processed
    
    def __len__(self):
        return len(self.processed_data)
    
    def __getitem__(self, idx):
        return self.processed_data[idx]

def collate_fn(batch):
    src_seqs = [item['src'] for item in batch]
    tgt_seqs = [item['tgt'] for item in batch]
    alignments = [item['alignment'] for item in batch]
    
    # Pad sequences
    src_max_len = max(len(seq) for seq in src_seqs)
    tgt_max_len = max(len(seq) for seq in tgt_seqs)
    
    src_padded = []
    tgt_padded = []
    tgt_input_padded = []
    alignment_padded = []
    
    for i in range(len(batch)):
        src_seq = src_seqs[i] + [0] * (src_max_len - len(src_seqs[i]))
        tgt_seq = tgt_seqs[i] + [0] * (tgt_max_len - len(tgt_seqs[i]))
        tgt_input_seq = tgt_seq[:-1] + [0] * (tgt_max_len - len(tgt_seq))
        
        # Pad alignment matrix
        alignment = alignments[i]
        padded_alignment = np.zeros((src_max_len, tgt_max_len - 1), dtype=np.float32)
        padded_alignment[:alignment.shape[0], :alignment.shape[1]] = alignment
        
        src_padded.append(src_seq)
        tgt_padded.append(tgt_seq[1:])  # Remove <sos> for target
        tgt_input_padded.append(tgt_input_seq)  # Remove <eos> for input
        alignment_padded.append(padded_alignment)
    
    return {
        'src': torch.LongTensor(src_padded),
        'tgt_input': torch.LongTensor(tgt_input_padded),
        'tgt_output': torch.LongTensor(tgt_padded),
        'alignment': torch.FloatTensor(np.array(alignment_padded))
    }

def train_model(model, train_loader, val_loader, num_epochs, device, learning_rate=0.0001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_trans_loss = 0
        total_align_loss = 0
        
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for batch_idx, batch in enumerate(progress_bar):
            src = batch['src'].to(device)
            tgt_input = batch['tgt_input'].to(device)
            tgt_output = batch['tgt_output'].to(device)
            alignment = batch['alignment'].to(device)
            
            optimizer.zero_grad()
            
            src_mask, tgt_mask = model.create_masks(src, tgt_input)
            model_output, cross_attention_weights = model(src, tgt_input, src_mask, tgt_mask)
            
            total_loss_batch, trans_loss_batch, align_loss_batch = calculate_total_loss(
                model_output, tgt_output, cross_attention_weights, alignment
            )
            
            total_loss_batch.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            # Clear cache periodically
            if batch_idx % 100 == 0:
                torch.cuda.empty_cache()
            
            total_loss += total_loss_batch.item()
            total_trans_loss += trans_loss_batch.item()
            total_align_loss += align_loss_batch.item()
            
            progress_bar.set_postfix({
                'Loss': f'{total_loss_batch.item():.4f}',
                'Trans': f'{trans_loss_batch.item():.4f}',
                'Align': f'{align_loss_batch.item():.4f}'
            })
            
        avg_train_loss = total_loss / len(train_loader)
        avg_trans_loss = total_trans_loss / len(train_loader)
        avg_align_loss = total_align_loss / len(train_loader)
        
        print(f'Epoch {epoch+1} - Train Loss: {avg_train_loss:.4f}, Trans: {avg_trans_loss:.4f}, Align: {avg_align_loss:.4f}')
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                src = batch['src'].to(device)
                tgt_input = batch['tgt_input'].to(device)
                tgt_output = batch['tgt_output'].to(device)
                alignment = batch['alignment'].to(device)
                
                src_mask, tgt_mask = model.create_masks(src, tgt_input)
                model_output, cross_attention_weights = model(src, tgt_input, src_mask, tgt_mask)
                
                total_loss_batch, _, _ = calculate_total_loss(
                    model_output, tgt_output, cross_attention_weights, alignment
                )
                val_loss += total_loss_batch.item()
        
        avg_val_loss = val_loss / len(val_loader)
        print(f'Epoch {epoch+1} - Val Loss: {avg_val_loss:.4f}')
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), '/kaggle/working/best_model.pth')
            print(f'New best model saved with val loss: {best_val_loss:.4f}')
        
        scheduler.step()
        print()

def main():
    torch.cuda.empty_cache()
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    dataset = TranslationDataset('/kaggle/input/kn-en-samanantar/samanantar_kn_translation_training (3).json')
    
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)
    
    src_vocab_size = len(dataset.src_vocab)
    tgt_vocab_size = len(dataset.tgt_vocab)
    
    print(f'src vocab size: {src_vocab_size}')
    print(f'target vocab size: {tgt_vocab_size}')
    
    model = EnglishKannadaTransformer(
        src_vocab_size=src_vocab_size,
        tgt_vocab_size=tgt_vocab_size,
        d_model=256,
        n_heads=8,
        n_encoder_layers=3,
        n_decoder_layers=3,
        d_ff=1024,
        max_seq_len=512,
        dropout=0.1
    )
    
    print(f'model init w {sum(p.numel() for p in model.parameters())} params')
    
    train_model(model, train_loader, val_loader, num_epochs=5, device=device)
    
    torch.save(model.state_dict(), '/kaggle/working/final_model.pth')
    
    torch.save({
        'src_vocab': dataset.src_vocab,
        'tgt_vocab': dataset.tgt_vocab,
        'src_word2idx': dataset.src_word2idx,
        'tgt_word2idx': dataset.tgt_word2idx
    }, '/kaggle/working/vocabularies.pth')
    
    print('Training completed!')

if __name__ == '__main__':
    main()

Using device: cuda
Source vocabulary size: 174304
Target vocabulary size: 418295
Model initialized with 264736759 parameters


Epoch 1/5:  27%|██▋       | 10735/39862 [25:25<1:10:08,  6.92it/s, Loss=9.6147, Trans=9.6126, Align=0.0021]  

In [23]:
from IPython.display import FileLink
FileLink(r'best_model.pth.zip')


/kaggle/working/best_model.pth.zip

In [27]:
from IPython.display import FileLink
FileLink(r'final_model.pth')


/kaggle/working/final_model.pth